In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2022-02-10 02:37:48,-46.1927,168.8643,75.47,29,16,8.25
1,1,Punta Arenas,CL,2022-02-10 02:35:48,-53.1500,-70.9167,46.51,57,40,24.16
2,2,Kapaa,US,2022-02-10 02:34:18,22.0752,-159.3190,78.78,70,75,6.91
3,3,Busselton,AU,2022-02-10 02:37:48,-33.6500,115.3333,84.04,36,5,3.02
4,4,Umm Lajj,SA,2022-02-10 02:37:49,25.0213,37.2685,66.22,59,0,13.69


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
#Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
#Get the maximum temperature
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
#Get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]

#Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [23]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2022-02-10 02:37:48,-46.1927,168.8643,75.47,29,16,8.25
2,2,Kapaa,US,2022-02-10 02:34:18,22.0752,-159.3190,78.78,70,75,6.91
3,3,Busselton,AU,2022-02-10 02:37:48,-33.6500,115.3333,84.04,36,5,3.02
5,5,Kavieng,PG,2022-02-10 02:37:49,-2.5744,150.7967,83.21,75,32,6.98
17,17,Bluff,NZ,2022-02-10 02:37:53,-46.6000,168.3333,75.79,84,45,12.62
24,24,Itarema,BR,2022-02-10 02:37:56,-2.9248,-39.9167,77.31,78,14,7.87
25,25,Atuona,PF,2022-02-10 02:37:56,-9.8000,-139.0333,78.58,77,11,17.00
26,26,Panguna,PG,2022-02-10 02:37:56,-6.3164,155.4848,78.30,78,68,3.42
31,31,Hilo,US,2022-02-10 02:37:58,19.7297,-155.0900,76.73,73,20,6.91
33,33,Hithadhoo,MV,2022-02-10 02:37:59,-0.6000,73.0833,81.88,73,33,11.18


In [24]:
preferred_cities_df.count()

City_ID       118
City          118
Country       118
Date          118
Lat           118
Lng           118
Max Temp      118
Humidity      118
Cloudiness    118
Wind Speed    118
dtype: int64

In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mataura,NZ,75.47,-46.1927,168.8643,
2,Kapaa,US,78.78,22.0752,-159.3190,
3,Busselton,AU,84.04,-33.6500,115.3333,
5,Kavieng,PG,83.21,-2.5744,150.7967,
17,Bluff,NZ,75.79,-46.6000,168.3333,
24,Itarema,BR,77.31,-2.9248,-39.9167,
25,Atuona,PF,78.58,-9.8000,-139.0333,
26,Panguna,PG,78.30,-6.3164,155.4848,
31,Hilo,US,76.73,19.7297,-155.0900,
33,Hithadhoo,MV,81.88,-0.6000,73.0833,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [27]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [28]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEC6JT09FD64MBxnnjMRwyiUeLZo1dEwv6ewQb5hXQpb1xShCW4afbPjpj7ZP_fAH-os5E0XVM9xmrkcJ3eJRFNOMhSGW7ajeroxJja8PBFN5LR2DnYQHrFckNrXWGt5pAjrDhO84rVZh4vq4xX7M9lEfHcQogZNWB_ec1tfCMerYv3Cz-g9iRb87Gz0dB0JrlJm2REnCj1gIp1jnHx0DCL1B3PAg0e0mKHPyfeetrFMxAMScvAl0IAmyWgdqX78KNaojrF3p6eINFxXB17cqO8SZVbTPov_dADGuVobOPw_G7qYLkCYfsuTZ2qSFMHwB0JIszTf0Xk_z2exF8ktadnVkB3weGb-MHRHvUy74jFmwIjsBls4mAi5Z5yFgvzqX1vbm5X2WDxI5Nbwk6-ZiC8fzUfKwqOS_PtpmexQMcJGPZ32L0aZvImg',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

IndexError: list index out of range

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
#Add your VacationPy.ipynb file to your World_Weather_Analysis GitHub repository.

In [31]:
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")

IndentationError: unexpected indent (Temp/ipykernel_9840/1383625399.py, line 2)

In [32]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [35]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [36]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [37]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))